# GGR376 LAB 5.1: Creating thematic maps using Python (10 marks)

### Exploratory data analysis (EDA)
In week 11 we discussed the importance of EDA for providing insights into data. Python and Jupyter notebooks provide an excellent tool for EDA, allowing the analyst to i) manipulate and visualize data comprehensively and ii) systematically document their workflow for other specialists to replicate. This capability empowers analysts to understand data structure and uncover patterns and outliers effectively. Such insights are valuable for informing decision-making around future analyses and visualization.

Let's start by exploring the population density by census tract data we used in Lab 1. We will use the [geopandas](https://geopandas.org/en/stable/docs/user_guide.html) library which is an extension of the data processing library, **pandas**, that allows us to work with spatial data. Data are stored as a **GeoDataFrame**: a tabular structure with a geometry field containing information aboout geometry type (e.g., point, line, or polygon) and location (e.g., coordinates).

We will also use the [matplot](https://matplotlib.org/stable/users/index.html) library to visualize the distribution of the data and plot different chart types.


As you progress through the notebook, look out for headings labelled **Your turn**. These sections are designed for you to apply concepts from the lecture and worksheet. In each **Your turn** section, you are expected to write code to achieve a specified outcome. To earn full marks, your code must include clear comments to demonstrate your understanding of concepts and code functionality. 

In [ ]:
# Import the libraries we will be using
import geopandas as gpd # 'as' provides an alias for the library that can be referenced in other code cells
import matplotlib.pyplot as plt

In [ ]:
# Read in the population shapefile from the 'data' folder and store as a geodataframe
torontopop21 = gpd.read_file('data/Toronto_Pop_CT2021_Python.shp')

# View the top five rows of data from the geodataframe to get a quick idea of the data structure and attributes
torontopop21.head()

In Python, **methods** are functions that are associated with objects. In this instance we are interested in functions associated with the **geodataframe object**.

Below are some useful Geopandas methods that allow us to get a sense of our data by exploring basic statistics and properties, such as data types, summary values, and geographic extent. 

```python

# Get the geometry type of the data
geodataframename.geom_type

# View and change coordinate reference systems
geodataframename.crs
geodataframename = torontopop21.to_crs(epsg=4326)

# Check for missing data
geodataframename.isnull().sum()

# Calculate summary statistics of attribute fields
geodataframename.describe()

# For geometries, you can check bounding box, area, length, etc.
geodataframename['area'] = geodataframename.geometry.area

```

*Tip:*
- *When retrieving an attribute value such as a geometry type or CRS, parentheses are not required (e.g., .crs). However, when calling a method that performs some computation, such as returning a table of summary values, parentheses are required (e.g., .describe() )*


Let's put some of those exploratory methods into action!

In [ ]:
# View coordinate reference system
torontopop21.crs

The shapefile uses the Canada Albers Equal Area Conic projection to **preserve area** within regions in Canada. It represents the Earth's surface on a flat, 2-D plane with units in metres (eastings and northings).

In [ ]:
# View some basic statistics about the population density column
torontopop21['PopDens21'].describe()

There are 583 census tracts with a mean population density of 8468.2 people per sq km. 

### Your turn (1 mark):
In the following cell, return the summary statistics for all fields in the geodataframe

### Exploratory visualizations
Now we have an idea about the data structure, we can create some exploratory visualizations. 

The [.plot()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html) method in geopandas allows us to create a map of a GeoDataFrame. In its simplest form **i.e. geodataframename.plot()**, a plot of geometries will be returned. However, it's possible to edit the output by adding parameters into the parentheses. For example, you can specify a field to plot data values, add a legend, change colour scheme, or change the plot type from the default map to an alternative chart such as a boxplot or histogram. Take time to explore the linked documentation to understand the different options.

Although we call the .plot() method on a geodataframe, it actually leverages the matplotlib library (remember we gave this library the alias 'plt' when we imported it in cell 1). It is therefore possible to combine matplotlib functions with the output to add a title, axes labels etc. 

There are several libraries that allow us to create charts. It is possible to use functions within the matplotlib library to create a [range of charts](https://matplotlib.org/stable/plot_types/index) such as data distribution charts (e.g., histograms, boxplots), statistical plots (e.g., scatter plots), and 3D charts (e.g., surface plots). The pandas library can also leverage matplotlib to create [non-spatial charts](https://pandas.pydata.org/docs/user_guide/visualization.html).


### Your turn (1 mark):
In the following cell, create a basic plot of census tract polygons

In [ ]:
# Use the geopandas .plot() method to plot an unclassified choropleth and see what the raw data show
torontopop21.plot(column='PopDens21', # Plot data for popn density field
                  legend=True, # Add legend
                  legend_kwds = {'label': "Population density by census tract (2021)"}) # Add legend options, in this case add a label

# Use matplotlib function to add title for plot
plt.title('Population density by census tract, Toronto (2021)')

# Add axes labels
plt.xlabel('Eastings [m]')
plt.ylabel('Northings [m]')

# Display plot
plt.show()

The **unclassified choropleth** provides a preliminary visual representation of the general distribution of the data, and has the potential to reveal outliers or natural clusters in the data. Let's also look at a histogram of the data to inform the classification process.


In [ ]:
# Use pandas .hist() method to plot a histogram and explore the data further
torontopop21['PopDens21'].hist()

# Alternative approaches to create a histogram
# matplotlib
#plt.hist(torontopop21['PopDens21'])

# geopandas
#torontopop21['PopDens21'].plot(kind='hist')

plt.title('Distribution of population density by census tracts in Toronto, 2021')
plt.xlabel('Population density [people per square km]')
plt.ylabel('Number of census tracts')
plt.show()

Both the unclassified choropleth and histogram indicate a **positive skew** in the data. 

It seems there's also a small number of CTs with a particularly high population density; the locations of which can be inferred from the map. We can quickly check the rows wth the highest population density values to see if there's anything that looks out of the ordinary with these outliers.

In [ ]:
# Sort the data by population density in descending order and get the top 5 rows
top_5_density = torontopop21.nlargest(5, 'PopDens21')

print("Top 5 rows with the highest population density:")
print(top_5_density)

### Choropleth map
There appears to be one CT with a particularly high population count but this is consistent with data recorded in 2016 so let's assume the value is valid.

Next we will create a classified choropleth map. Given the positive skew of the data, equal intervals are not appropriate as this would lead to an unequal distribution of observations across the classes and grouping of dissimilar values. We will therefore use **quantiles**. Quantiles divide data into equal portions based on frequency of data points and often handle skewed distributions more effectively by ensuring each group contains a similar number of observations.

In [ ]:
# Create a choropleth map with five quantiles (quintiles)
torontopop21.plot(column= 'PopDens21',
                  legend=True,
                  cmap='BuPu', # Specify a colour map
                  scheme="quantiles", k=5, # Classify data by quintiles
                  legend_kwds={'loc': 'upper left', # Choose point on legend box to use for anchoring
                               'bbox_to_anchor':(1,1) # Anchor legend outside plot where (0,0) = bottom-left and (1,1) = top-right
                              }
                 ).set_axis_off(); # Turn axes lines and labels off

So far we have created static maps, but introducing interactivity can enhance our understanding of the data by allowing us to zoom/pan into areas of interest and  to click on map features to retrieve associated information. We can create interactive maps using the [.explore()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) method which leverages a library called **folium** to create a webmap.

In [ ]:
# Use the .explore() method to create an interactive map
torontopop21.explore("PopDens21", 
                     legend=False,
                     cmap='BuPu',
                     scheme="quantiles", k=5)

# Hover mouse over census tracts to see a pop-up of attribute values

### Your turn (3 marks):
Using the code above for reference, create the following visualizations:
1. box plot of population density data for 2016 and 2021 (hint: this is possible using [pandas](https://pandas.pydata.org/docs/user_guide/visualization.html#visualization-box), [matplotlib](https://www.geeksforgeeks.org/box-plot-in-python-using-matplotlib/), or [geopandas](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html) libraries. To add multiple fields as parameters, double square brackets must be used e.g., torontopop21[['PopDens16', 'PopDens21']])
2. static choropleth of 2016 population density data classified with 4 [natural breaks](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html)
3. [interactive choropleth](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html) of 2016 population density data using the same classification and colour scheme as the static map

As a minimum, you are expected to change the [colour scheme](https://matplotlib.org/stable/gallery/color/colormap_reference.html), update the number of classes, and add comments to describe what each section of code does. **Be creative with your updates!**

In [ ]:
# Start by creating and calculating a new field for population density
# Calculate area in square kilometers from the geometry field
torontopop21['Area_km2'] = torontopop21['geometry'].area / 10**6  # convert area to square kilometers

# Calculate population density and assign it to a new column 'PopDens16'
torontopop21['PopDens16'] = torontopop21['Popn16'] / torontopop21['Area_km2']

torontopop21.head()

In [ ]:
# 1. Box plot
# Insert your code here and run the cell...



In [ ]:
# Static map
# Insert your code here and run the cell...



In [ ]:
# Interactive map
# Insert your code here and run the cell...



### Proportional symbols maps
Next we will aim to create a proportional symbols map of percentage of population change between 2021 and 2016. As a reminder, proportional symbol maps use symbols where the size of the symbol is proportional to the value of the data being represented. A greater percentage of change will therefore be represented by a larger symbol where its size is relative to the value on a continuous (unclassified) data scale.

Let's start by creating a **simple proportional symbols map of raw population counts for 2021**. This will require some data preparation to first create a geodataframe of points and to second join our new points geodataframe to our original data. Take time to explore the methods required for this:

- [geopandas .centroid()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.centroid.html)
- [pandas .join()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)


In [ ]:
# Extract the centroids of census tract polygons as a new geodataframe so that the points can be used to place symbols
tractcentroids = gpd.GeoDataFrame(torontopop21.centroid, columns=['geometry',])

# Create non-spatial subset of original data and retain desired attribute values
popdens = torontopop21[['CTNAME','Popn16', 'Popn21', 'PopDens16', 'PopDens21']].copy()

# Join non-spatial data with new points data
tractcentroids = tractcentroids.join(popdens)

# Show top five rows of new point data
tractcentroids.head()


For the proportional symbols map, let's add a layer showing the census tract outlines. To create and manage **multiple plots within a single figure**, we can use the matplotlib [.subplots()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) function.

In [ ]:
# Set the symbol size to the 2021 absolute population value. We divide by 100 here to make the the symbol size more appropriate for the display
symbol = (tractcentroids['Popn21']/100)

# Create a subplot to plot both layers
fig, ax = plt.subplots(figsize=(10,8)) # 'fig' refers to the canvas area and 'ax' to the sublot within the canvas. Here we set the subplot size to accommodate our symbols (without setting figure size, the default canvas size will be used)

# Plot a simple basemap of census tract polygons
torontopop21.plot(ax = ax, # Draw geodataframe on subplot ax
                  facecolor='lightgray',
                  edgecolor='gray')

# Add proportional symbols for population counts
tractcentroids.plot(ax = ax, # Draw point geodataframe on same subplot ax
                     markersize=symbol, # Set marker to our symbol size previously calculated
                     color='blue',
                     label = '2021 Population by census tract') # Specify label for legend

# Add legend to subplot (legend includes all entries with a label)
ax.legend()

### Your turn (4 marks):
Using the code above for guidance, create the following visualization:
- A proportional symbols map of percentage change of raw population between 2016 and 2021

The following three steps will help you achieve this:
1. Create a new field `PcChange` to store percentage of population change for your **geodataframe of points** where:

$$ \text{Percentage Change} = \frac{\text{New Value} - \text{Old Value}}{\text{Old Value}} \times 100 $$

2. Create two new fields for your **geodataframe of points** storing values for percentage increase and percentage decrease

3. Use the matplotlib [.subplots()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) function to plot percentage increase and percentage decrease on the same subplot




In [ ]:
# 1. Calculate percentage change in population and store value in new field 'PcChange'
# Insert your code here and run the cell...



Next we will create two new fields to visualize on the same subplot. Using the **Python handbook**, see if you can work out what is happening in the following cell.

**Add comments using the # symbol to explain what each row of code does.**

In [ ]:
# 2. Add and populate fields for percentage increase and decrease

### ADD COMMENTS to each line of code below and run the cell...

tractcentroids['PcInc'] = 0.0
tractcentroids['PcDec'] = 0.0

for i in range(0, len(tractcentroids)):
    pcvalue = tractcentroids.loc[i, 'PcChange']
    if pcvalue > 0: 
        tractcentroids.loc[i,'PcInc'] = pcvalue
    else: 
        tractcentroids.loc[i,'PcDec'] = (pcvalue*-1)

tractcentroids.head()


In [ ]:
# 3. Create proportional symbols plot of percentage change in Toronto between 2016 and 2021
# Insert your code here and run the cell...



Once you have completed the lab, export your notebook by selecting File > Print Preview > Save as PDF

Upload your PDF to Part 1 of the lab in Quercus.